## Steps for Building Network
[Step 1](#step1): Unzip image dataset and check out train, validation and test files.

[Step 2](#step2): Show image and json files in train & validation dataset.

[Step 3](#step3): Initialize features(input) and labels(output) from images and json list.

- read images from train/validation/test path.
- read labels from train/validation json file.
- resize and normalize images.
- get batch and return feature_batch and label_batch.

[Step 4](#step4): Build convolutional network, return training accuracy and training loss.

[Step 5](#step5): Train on steps = 20000.

[Step 6](#step6): Train on full dataset.
- epoch x, batch x, training loss, validation accuracy, evluation accuracy

[Step 7](#step7): Test and write json submit file.

<a id='step1'></a>
## Step 1: Unzip image dataset and check out train, validation and test files.

In [1]:
import os, zipfile

train_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_train_20170904.zip'
validation_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_validation_20170908.zip'
test_a_path = 'E:/ai_challenger/scene classification/dataset/ai_challenger_scene_test_a_20170922.zip'
extract_path = 'E:/ai_challenger/scene classification/dataset'

def unzip(zipfile_path, extract_path, zipfile_name):
    zipfile = zipfile.ZipFile(zipfile_path, 'r')
    print('Extracting {} ...'.format(zipfile_name))
    zipfile.extractall()
    zipfile.close()
    print('{} has been extracted.'.format(zipfle_name))

if os.path.exists(extract_path):
    print('Found extraced dataset')
else:
    unzip(train_path, extract_path, 'training dataset')
    unzip(validation_path, extract_path, 'validation dataset')
    unzip(test_a_path, extract_path, 'test dataset')

Found extraced dataset


<a id='step2'></a>
## Step 2: Show image and json files in train & validation dataset.

In [2]:
import json
import glob
from scipy.misc import imread, imresize, imsave
import numpy as np

train_features_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_images_20170904'
train_labels_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_annotations_20170904.json'
validation_features_path =r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_images_20170908'
validation_labels_path =r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_validation_20170908\scene_validation_images_20170908\scene_validation_annotations_20170908.json'
test_a_features_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_test_a_20170922\scene_test_a_images_20170922'

# Show train label list
with open(train_labels_path, 'r') as f:
    train_label_list = json.load(f)
    print(train_label_list[:10])
    train_dict = {}
    for image in train_label_list:
        train_dict[image['image_id']] = int(image['label_id'])
    print('\n')
    print(len(train_dict))     

# Show train features list resulting out of memory...

[{'image_url': 'https://n1-q.mafengwo.net/s1/M00/6B/72/wKgBm04Wc5WzFXU0AAHf09bdpiY84.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90', 'image_id': '79f993ae0858ae238b22968c5934d1ddba585ae4.jpg', 'label_id': '66'}, {'image_url': 'http://news.sogou.com/', 'image_id': 'e963208fe9e90df0c385f7367bcdb6d0d5d0b165.jpg', 'label_id': '61'}, {'image_url': 'http://img2.fawan.com/2016/12/30/e967f93e7713c57cd2b00b832dd6091a_500x-_90.jpg', 'image_id': '02df5ecbf7c749ccc9d833f129bbd5d9837940ce.jpg', 'label_id': '64'}, {'image_url': 'https://b1-q.mafengwo.net/s1/M00/F2/C9/wKgBm04Wx3a-gk2FAAKbPKX7E9w91.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90', 'image_id': '5620eb385b7567fb087813cf5233b5ceecdeeca3.jpg', 'label_id': '31'}, {'image_url': 'http://news.sogou.com/', 'image_id': 'f8b4d42001a562fc63b9b39c02531661c0e236ca.jpg', 'label_id': '19'}, {'image_url': 'http://www.user2.jqw.com/2014/01/06/1347666/product/b201401072000291460.JPG', 'image_id': '57e7eb438670a4519041dab1482f2594a92f8a09.jpg', 'label_id': '11'}, {'imag

<a id='step3'></a>
## Step 3: Initialize features(input) and labels(output) from images and json list.

In [3]:
import json
from scipy.misc import imread, imresize
import numpy as np
import os

class initialize(object):
    # Get image-label list for train and validation
    def __init__(self, feature_path, label_path):
        self.image_label_dict = {}
        with open(label_path, 'r') as f:
            label_list = json.load(f)
        for image in label_list:
            self.image_label_dict[image['image_id']] = int(image['label_id'])
        self.start = 0
        self.end = 0
        self.length = len(self.image_label_dict) # number of feature images
        self.image_name = list(self.image_label_dict.keys())
        self.feature_path = feature_path
    
    # Read image in feature path, resize and normalize to [-1, 1]
    def get_image(self, image_path, image_size):
        image = imread(image_path)
        image = imresize(image, [image_size, image_size])       
        image = np.array(image).astype(np.float32)
        image = 2 * (image - np.min(image)) / np.ptp(image) - 1
        return image
    
    # Get feature and label batch
    def get_batch(self, batch_size, image_size):
        self.start = self.end
        if self.start >= self.length:
            self.start = 0
        batch_feature = []
        batch_label = []
        index = self.start
        while len(batch_feature) < batch_size:
            if index >= self.length:
                index = 0
            i_image_path = os.path.join(self.feature_path, self.image_name[index])
            i_image = self.get_image(i_image_path, image_size)
            i_label = self.image_label_dict[self.image_name[index]]
            batch_feature.append(i_image)
            batch_label.append(i_label)
            index += 1
        self.end = index
        return batch_feature, batch_label

<a id='step4'></a>
## Step 4: 16 convolutional layers with batch normalization.

In [4]:
import tensorflow as tf

def conv_layer(input_layer, filters, strides, is_training):
    layer = tf.layers.conv2d(
        inputs=input_layer, 
        filters=filters, 
        kernel_size=3,
        strides=strides, 
        padding='same', 
        activation=None,
        kernel_initializer=tf.truncated_normal_initializer()
    )
    
    layer = tf.layers.batch_normalization(
        inputs=layer, 
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        training=is_training
    )
    
    layer = tf.nn.relu(layer)
    return layer

def fully_connected(input_layer, num_units, is_training):
    layer = tf.layers.dense(input_layer, num_units, use_bias=False, activation=None)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    return layer

def conv_network(feature, label, num_class, image_size, keep_prob, is_training):
    input_layer = tf.reshape(feature, [-1, image_size, image_size, 3])
    
    # 16 conv layers with 64, 128, 256, 512 filters.
    layer = conv_layer(input_layer, 64, 1, is_training)
    layer = conv_layer(layer, 64, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 128, 1, is_training)
    layer = conv_layer(layer, 128, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    shape = layer.get_shape().as_list()
    layer = tf.reshape(layer, shape=[-1, shape[1]*shape[2]*shape[3]])
    layer = fully_connected(layer, 800, is_training)
    layer = tf.nn.dropout(layer, keep_prob)
    logits = tf.layers.dense(layer, 80)
    output = tf.sigmoid(logits)
    
    # Loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label))
    
    return logits, loss


In [5]:
def train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step):
    
    train = initialize(train_feature_path, train_label_path)
        
    feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name='feature')
    label = tf.placeholder(tf.float32, shape=[None], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    one_hot_label = tf.one_hot(indices=tf.cast(label, tf.int32), depth=80)
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    logits, loss = conv_network(feature, one_hot_label, num_class, image_size, keep_prob, is_training)
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
       
    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_path)
        
        if ckpt and ckpt.model_checkpoint_path:
            print('Restore the model from checkpoint {}.'.format(ckpt.model_checkpoint_path))
            start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        else:
            sess.run(tf.global_variables_initializer())
            start_step = 0
            print('Start training from new start.')
        
        for steps in range(start_step, start_step + max_step):
            train_feature_batch, train_label_batch = train.get_batch(batch_size, image_size)
            #print(is_training, feature, label)
            sess.run(train_opt, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: True})
                
            if steps % 10 == 0:
                train_accuracy = sess.run(accuracy, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                train_loss = sess.run(loss, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                print('Step {}'.format(steps),
                      'Training Accuracy {:.3f}...'.format(train_accuracy),
                      'Training Loss {:.3f}...'.format(train_loss),
                     ) 
            if steps % 1000 == 0:
                saver.save(sess, checkfile, global_step=steps)
                print('Writing checkpoing at step {}'.format(steps))

        print('Training completed.')

# image pathes
train_feature_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_images_20170904'
train_label_path = r'E:\ai_challenger\scene classification\dataset\ai_challenger_scene_train_20170904\scene_train_annotations_20170904.json'
checkpoint_path = './checkpoint/'
checkfile = './checkpoint/model.ckpt'

num_class = 80
batch_size = 64
image_size = 64
max_step = 10000
learning_rate =0.002

train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step)

Start training from new start.
Step 0 Training Accuracy 0.016... Training Loss 792491200.000...
Writing checkpoing at step 0
Step 10 Training Accuracy 0.000... Training Loss 379.534...
Step 20 Training Accuracy 0.016... Training Loss 24.364...
Step 30 Training Accuracy 0.016... Training Loss 7.035...
Step 40 Training Accuracy 0.000... Training Loss 5.493...
Step 50 Training Accuracy 0.031... Training Loss 5.455...
Step 60 Training Accuracy 0.000... Training Loss 5.235...
Step 70 Training Accuracy 0.016... Training Loss 4.674...
Step 80 Training Accuracy 0.031... Training Loss 5.072...
Step 90 Training Accuracy 0.062... Training Loss 4.950...
Step 100 Training Accuracy 0.016... Training Loss 5.491...
Step 110 Training Accuracy 0.016... Training Loss 5.201...
Step 120 Training Accuracy 0.016... Training Loss 4.916...
Step 130 Training Accuracy 0.031... Training Loss 6.100...
Step 140 Training Accuracy 0.062... Training Loss 4.790...
Step 150 Training Accuracy 0.047... Training Loss 4.273

Step 1370 Training Accuracy 0.188... Training Loss 3.181...
Step 1380 Training Accuracy 0.125... Training Loss 3.536...
Step 1390 Training Accuracy 0.219... Training Loss 3.283...
Step 1400 Training Accuracy 0.125... Training Loss 3.369...
Step 1410 Training Accuracy 0.203... Training Loss 3.192...
Step 1420 Training Accuracy 0.156... Training Loss 3.543...
Step 1430 Training Accuracy 0.141... Training Loss 3.913...
Step 1440 Training Accuracy 0.188... Training Loss 3.329...
Step 1450 Training Accuracy 0.125... Training Loss 3.430...
Step 1460 Training Accuracy 0.219... Training Loss 3.477...
Step 1470 Training Accuracy 0.141... Training Loss 3.524...
Step 1480 Training Accuracy 0.172... Training Loss 3.301...
Step 1490 Training Accuracy 0.250... Training Loss 3.220...
Step 1500 Training Accuracy 0.109... Training Loss 3.549...
Step 1510 Training Accuracy 0.203... Training Loss 3.212...
Step 1520 Training Accuracy 0.219... Training Loss 3.327...
Step 1530 Training Accuracy 0.094... Tra

Step 2740 Training Accuracy 0.188... Training Loss 3.131...
Step 2750 Training Accuracy 0.250... Training Loss 3.130...
Step 2760 Training Accuracy 0.234... Training Loss 3.135...
Step 2770 Training Accuracy 0.203... Training Loss 2.904...
Step 2780 Training Accuracy 0.266... Training Loss 2.922...
Step 2790 Training Accuracy 0.281... Training Loss 2.834...
Step 2800 Training Accuracy 0.281... Training Loss 2.615...
Step 2810 Training Accuracy 0.234... Training Loss 3.364...
Step 2820 Training Accuracy 0.297... Training Loss 2.876...
Step 2830 Training Accuracy 0.250... Training Loss 2.864...
Step 2840 Training Accuracy 0.297... Training Loss 2.863...
Step 2850 Training Accuracy 0.250... Training Loss 2.902...
Step 2860 Training Accuracy 0.188... Training Loss 3.096...
Step 2870 Training Accuracy 0.328... Training Loss 2.480...
Step 2880 Training Accuracy 0.328... Training Loss 2.774...
Step 2890 Training Accuracy 0.188... Training Loss 2.915...
Step 2900 Training Accuracy 0.250... Tra

Step 4100 Training Accuracy 0.328... Training Loss 2.377...
Step 4110 Training Accuracy 0.406... Training Loss 2.315...
Step 4120 Training Accuracy 0.328... Training Loss 2.680...
Step 4130 Training Accuracy 0.297... Training Loss 2.538...
Step 4140 Training Accuracy 0.172... Training Loss 3.289...
Step 4150 Training Accuracy 0.312... Training Loss 2.801...
Step 4160 Training Accuracy 0.297... Training Loss 2.498...
Step 4170 Training Accuracy 0.312... Training Loss 2.651...
Step 4180 Training Accuracy 0.375... Training Loss 2.597...
Step 4190 Training Accuracy 0.266... Training Loss 2.599...
Step 4200 Training Accuracy 0.297... Training Loss 2.569...
Step 4210 Training Accuracy 0.312... Training Loss 2.867...
Step 4220 Training Accuracy 0.250... Training Loss 2.718...
Step 4230 Training Accuracy 0.422... Training Loss 1.996...
Step 4240 Training Accuracy 0.234... Training Loss 2.903...
Step 4250 Training Accuracy 0.297... Training Loss 2.940...
Step 4260 Training Accuracy 0.438... Tra

Step 5470 Training Accuracy 0.469... Training Loss 2.080...
Step 5480 Training Accuracy 0.453... Training Loss 2.037...
Step 5490 Training Accuracy 0.375... Training Loss 2.241...
Step 5500 Training Accuracy 0.406... Training Loss 2.218...
Step 5510 Training Accuracy 0.297... Training Loss 2.428...
Step 5520 Training Accuracy 0.328... Training Loss 2.561...
Step 5530 Training Accuracy 0.297... Training Loss 2.690...
Step 5540 Training Accuracy 0.391... Training Loss 2.476...
Step 5550 Training Accuracy 0.484... Training Loss 1.928...
Step 5560 Training Accuracy 0.500... Training Loss 2.073...
Step 5570 Training Accuracy 0.406... Training Loss 2.244...
Step 5580 Training Accuracy 0.438... Training Loss 2.275...
Step 5590 Training Accuracy 0.328... Training Loss 2.669...
Step 5600 Training Accuracy 0.359... Training Loss 2.196...
Step 5610 Training Accuracy 0.391... Training Loss 2.279...
Step 5620 Training Accuracy 0.375... Training Loss 2.299...
Step 5630 Training Accuracy 0.375... Tra

Step 6840 Training Accuracy 0.344... Training Loss 2.421...
Step 6850 Training Accuracy 0.359... Training Loss 2.128...
Step 6860 Training Accuracy 0.500... Training Loss 1.946...
Step 6870 Training Accuracy 0.359... Training Loss 2.323...
Step 6880 Training Accuracy 0.469... Training Loss 2.054...
Step 6890 Training Accuracy 0.484... Training Loss 2.076...
Step 6900 Training Accuracy 0.531... Training Loss 1.846...
Step 6910 Training Accuracy 0.500... Training Loss 1.575...
Step 6920 Training Accuracy 0.406... Training Loss 1.988...
Step 6930 Training Accuracy 0.375... Training Loss 2.135...
Step 6940 Training Accuracy 0.438... Training Loss 2.086...
Step 6950 Training Accuracy 0.469... Training Loss 2.102...
Step 6960 Training Accuracy 0.406... Training Loss 2.154...
Step 6970 Training Accuracy 0.469... Training Loss 1.865...
Step 6980 Training Accuracy 0.438... Training Loss 1.778...
Step 6990 Training Accuracy 0.484... Training Loss 1.863...
Step 7000 Training Accuracy 0.453... Tra

Step 8200 Training Accuracy 0.531... Training Loss 1.575...
Step 8210 Training Accuracy 0.516... Training Loss 1.833...
Step 8220 Training Accuracy 0.469... Training Loss 1.774...
Step 8230 Training Accuracy 0.484... Training Loss 1.781...
Step 8240 Training Accuracy 0.578... Training Loss 1.741...
Step 8250 Training Accuracy 0.562... Training Loss 1.685...
Step 8260 Training Accuracy 0.484... Training Loss 1.744...
Step 8270 Training Accuracy 0.391... Training Loss 2.015...
Step 8280 Training Accuracy 0.453... Training Loss 1.865...
Step 8290 Training Accuracy 0.578... Training Loss 1.579...
Step 8300 Training Accuracy 0.547... Training Loss 1.703...
Step 8310 Training Accuracy 0.453... Training Loss 1.647...
Step 8320 Training Accuracy 0.469... Training Loss 2.164...
Step 8330 Training Accuracy 0.453... Training Loss 1.518...
Step 8340 Training Accuracy 0.516... Training Loss 1.366...
Step 8350 Training Accuracy 0.547... Training Loss 1.589...
Step 8360 Training Accuracy 0.547... Tra

Step 9570 Training Accuracy 0.547... Training Loss 1.639...
Step 9580 Training Accuracy 0.516... Training Loss 1.335...
Step 9590 Training Accuracy 0.547... Training Loss 1.605...
Step 9600 Training Accuracy 0.625... Training Loss 1.317...
Step 9610 Training Accuracy 0.703... Training Loss 1.051...
Step 9620 Training Accuracy 0.469... Training Loss 1.618...
Step 9630 Training Accuracy 0.750... Training Loss 1.049...
Step 9640 Training Accuracy 0.484... Training Loss 1.409...
Step 9650 Training Accuracy 0.500... Training Loss 1.644...
Step 9660 Training Accuracy 0.625... Training Loss 1.274...
Step 9670 Training Accuracy 0.656... Training Loss 1.159...
Step 9680 Training Accuracy 0.562... Training Loss 1.625...
Step 9690 Training Accuracy 0.609... Training Loss 1.376...
Step 9700 Training Accuracy 0.703... Training Loss 1.169...
Step 9710 Training Accuracy 0.500... Training Loss 1.420...
Step 9720 Training Accuracy 0.578... Training Loss 1.420...
Step 9730 Training Accuracy 0.641... Tra

<a id='step5'></a>
## Step 5: With Xavier initialization

<a id='step6'></a>
## Step 6: Train on full dataset.